In [ ]:
# librerie
import os
import torch
import random
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import Subset
from data_augmentation import augment_data
import matplotlib.pyplot as plt
import collections
from torch.utils.data import random_split
import PIL
from PIL import Image, ImageDraw, ImageFont

In [3]:
# riproducibilità
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
# configurazione
image_size = 224 # dimensione a cui ridimensionare le immagini
noise_std = 0.2 # livello di rumore gaussiano da aggiungere ai dati augmentati
train_ratio, val_ratio = 0.7, 0.15  # percentuali per suddividere in train/val/test

In [ ]:
# prendo percorso base tramite os
base_dir = os.getcwd()
# percorso dataset  
data_dir = os.path.join(base_dir, 'data_histo') 
# percorso per salvare output
output_folder = os.path.join(base_dir, "shared_augmented_data")
# crea se non esiste
os.makedirs(output_folder, exist_ok=True) 

In [ ]:
# trasformazioni 
transform_base = transforms.Compose([
    transforms.Resize((image_size, image_size)),  # ridimensiona le immagini
    transforms.ToTensor(),                        # converte in tensore
])

In [ ]:
# caricamento dataset e applicazione trasformazioni
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform_base) 

In [ ]:
# definiamo le dimensioni di train/val/test, train 70%, val 15% e test 15%
total_size = len(full_dataset)                                           # numero totale di immagini
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
test_size = total_size - train_size - val_size                           # per evitare arrotondamenti

# suddivisione indici casuale per il train set ma riproducibile, 
# ci serve solo il train perchè applichiamo data augmentation solo su questo
train_indices, _, _ = random_split(
    list(range(total_size)), [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# recupero dei path (e delle classi) delle sole immagini di train
train_samples = [full_dataset.samples[i] for i in train_indices]

In [ ]:
# applicazione data augmentation, funzione in data_augemntation.py
print("Generazione dati augmentati...")
train_imgs, train_labels, _ = augment_data(
    samples=train_samples,
    class_names=full_dataset.classes,
    image_size=image_size,
    mode="balance",             # bilanciamento classi con oversampling
    add_noise=True,             # aggiunta di rumore gaussiano alle immagini
    noise_std=noise_std         # deviazione standard del rumore
)

Generazione dati augmentati...


In [ ]:
# salvataggio dati augmentati (immagini + etichette) per utilizzo futuro
save_path = os.path.join(output_folder, f"augmented_train_data_{noise_std}.pt")
torch.save({
    'images': train_imgs,
    'labels': train_labels
}, save_path)

print(f"Dati augmentati salvati in: {save_path}")

In [ ]:
# caricamento immagini augmentate salvate per poterne plottare qualche esempio da visualizzare
data = torch.load(save_path, map_location="cpu") 

images = data['images']
labels = data['labels']

# path dove salvare il plot con gli esempi
image_path = os.path.join(output_folder, f"augmented_images_sample_{noise_std}.png")
# nomi classi leggibili
pretty_classes = ['Adenocarcinoma', 'Benigno', 'Squamoso']

try:
    font = ImageFont.truetype("arial.ttf", size=20) # font per label
except:
    font = ImageFont.load_default()

# 3 immagini random
indices = random.sample(range(len(images)), 3)
imgs = []

for i in indices:
    img = images[i].detach().cpu().permute(1, 2, 0).numpy() # cambio formato con la permuta per matplot, da [C, H, W] 
                                                                # (formato tensori Pytorch) a [H, W, C] (formato compatibile con matplot) 
                                                            
    img = np.clip(img * 255, 0, 255).astype(np.uint8)           # no, de-normalizzazione perchè in con data_augmentation non normalizziamo
                                                                # solo conversione nel formato corretto per visualizzare l'immagine
    img_pil = Image.fromarray(img)                              # immagini PIL

    # canvas con spazio per testo
    width, height = img_pil.size
    canvas = Image.new("RGB", (width, height + 30), color=(255, 255, 255)) # camvas con 30 pixel in più in altezza per label
    canvas.paste(img_pil, (0, 0)) # inserisce immagine su canvas

    draw = ImageDraw.Draw(canvas)
    label = labels[i].item() # estrazione etichetta 
    class_name = pretty_classes[label] # leggibile

    bbox = draw.textbbox((0, 0), class_name, font=font) # centratura testo
    text_width = bbox[2] - bbox[0]
    text_x = (width - text_width) // 2
    draw.text((text_x, height + 5), class_name, fill=(0, 0, 0), font=font) # scrittura nome classe con colore testo nero (fill = (0,0,0))

    imgs.append(canvas)

# in una sola immagine
combined = Image.new("RGB", (width * 3, height + 30)) # griglia orizzontale per 3 immagini
for i, img in enumerate(imgs):
    combined.paste(img, (i * width, 0))

combined.save(image_path) # salvataggio
print(f"Salvata immagine random in: {image_path}")

Salvata immagine random in: c:\Users\noemi\Documents\GitHub\PCA_AE_histology\shared_augmented_data\augmented_images_sample_0.2.png
